In [1]:
# Some utility classes to represent a PDB structure

class Atom:
    """
    A simple class for an amino acid residue
    """

    def __init__(self, type):
        self.type = type
        self.coords = (0.0, 0.0, 0.0)

    # Overload the __repr__ operator to make printing simpler.
    def __repr__(self):
        return self.type

class Residue:
    """
    A simple class for an amino acid residue
    """

    def __init__(self, type, number):
        self.type = type
        self.number = number
        self.atoms = []

    # Overload the __repr__ operator to make printing simpler.
    def __repr__(self):
        return "{0} {1}".format(self.type, self.number)

class ActiveSite:
    """
    A simple class for an active site
    """

    def __init__(self, name):
        self.name = name
        self.residues = []

    # Overload the __repr__ operator to make printing simpler.
    def __repr__(self):
        return self.name


In [2]:
import glob
import os



def read_active_sites(dir):
    """
    Read in all of the active sites from the given directory.

    Input: directory
    Output: list of ActiveSite instances
    """
    files = glob.glob(dir + '/*.pdb')

    active_sites = []
    # iterate over each .pdb file in the given directory
    for filepath in glob.iglob(os.path.join(dir, "*.pdb")):

        active_sites.append(read_active_site(filepath))

    print("Read in %d active sites"%len(active_sites))

    return active_sites


def read_active_site(filepath):
    """
    Read in a single active site given a PDB file

    Input: PDB file path
    Output: ActiveSite instance
    """
    basename = os.path.basename(filepath)
    name = os.path.splitext(basename)

    if name[1] != ".pdb":
        raise IOError("%s is not a PDB file"%filepath)

    active_site = ActiveSite(name[0])

    r_num = 0

    # open pdb file
    with open(filepath, "r") as f:
        # iterate over each line in the file
        for line in f:
            if line[0:3] != 'TER':
                # read in an atom
                atom_type = line[13:17].strip()
                x_coord = float(line[30:38])
                y_coord = float(line[38:46])
                z_coord = float(line[46:54])
                atom = Atom(atom_type)
                atom.coords = (x_coord, y_coord, z_coord)

                residue_type = line[17:20]
                residue_number = int(line[23:26])

                # make a new residue if needed
                if residue_number != r_num:
                    residue = Residue(residue_type, residue_number)
                    r_num = residue_number

                # add the atom to the residue
                residue.atoms.append(atom)

            else:  # I've reached a TER card
                active_site.residues.append(residue)

    return active_site


def write_clustering(filename, clusters):
    """
    Write the clustered ActiveSite instances out to a file.

    Input: a filename and a clustering of ActiveSite instances
    Output: none
    """

    out = open(filename, 'w')

    for i in range(len(clusters)):
        out.write("\nCluster %d\n--------------\n" % i)
        for j in range(len(clusters[i])):
            out.write("%s\n" % clusters[i][j])

    out.close()


def write_mult_clusterings(filename, clusterings):
    """
    Write a series of clusterings of ActiveSite instances out to a file.

    Input: a filename and a list of clusterings of ActiveSite instances
    Output: none
    """

    out = open(filename, 'w')

    for i in range(len(clusterings)):
        clusters = clusterings[i]

        for j in range(len(clusters)):
            out.write("\nCluster %d\n------------\n" % j)
            for k in range(len(clusters[j])):
                out.write("%s\n" % clusters[j][k])

    out.close()


In [3]:
example_path="/Users/student/Documents/hw2-skeleton/data"

In [4]:
# this defintion read the entire data
test=read_active_sites(example_path)



Read in 136 active sites


In [5]:
len(test)

136

In [6]:
first=test[0]

In [7]:
first

46495

In [8]:
first.name

'46495'

In [9]:
first.residues

[ASP 165, ASP 167, SER 211, ARG 213, ASP 254, LYS 258, ASP 278]

In [10]:
first.residues[0].atoms

[N, CA, C, O, CB, CG, OD1, OD2]

In [11]:
first.residues[0].atoms[0]

N

In [12]:
first.residues[0].atoms[0].coords

(41.692, 10.964, 19.961)

In [15]:
def avg_coordinates(PDB):
    residues_list=[]
    atoms_list=[]
    for residue in PDB.residues:
        if residue == ['CA','N','C','CB']:
            residues_list.append(residue)
        for atoms in residue.atoms:
            atoms_list.append(atoms.coords)
            
    return atoms_list
    
    
    

In [17]:
# Convert the backbone coordinate list into a dataframe: 
import pandas as pd
import numpy as np
def convert_dataframe(List):
    backbone_coordinates = pd.DataFrame(np.array(List).reshape(len(List),3), columns = list("xyz"))
    return(backbone_coordinates)



In [18]:
def all_active_sites(test):
    for i in test:
        example_list=avg_coordinates(i)
        convert = convert_dataframe(example_list)
        one_active_site=pd.DataFrame(convert.mean(),columns=[i.name]).T
        yield one_active_site


In [19]:
mean_of_all_activesites = pd.concat(list(all_active_sites(test)))

In [43]:
mean_of_all_activesites.head()

,x,y,z
46495,43.935672,14.163776,25.155793
23812,97.470347,49.718286,26.240388
41729,21.226330,36.436375,12.912402
91911,-0.426087,-0.153887,36.409313
82212,60.954000,12.616917,48.877250


In [22]:
new_array=np.array(mean_of_all_activesites)

In [35]:
#new_array

In [21]:
len(mean_of_all_activesites)

136

In [32]:
#Caculating euclidean distance by pairwise: using scipy.spatial distance
from scipy.spatial import distance

#Y = distance.pdist(new_array, 'euclidean')
Y = distance.squareform(distance.pdist(new_array, 'euclidean'))

In [33]:
Y

array([[  0.        ,  64.2748854 ,  34.0834787 , ...,  19.7028291 ,
         78.83148657, 159.45596423],
       [ 64.2748854 ,   0.        ,  78.53148685, ...,  83.04319709,
         86.82621058, 215.35110349],
       [ 34.0834787 ,  78.53148685,   0.        , ...,  40.04646147,
         71.69224029, 160.129774  ],
       ...,
       [ 19.7028291 ,  83.04319709,  40.04646147, ...,   0.        ,
         91.75320237, 141.24777578],
       [ 78.83148657,  86.82621058,  71.69224029, ...,  91.75320237,
          0.        , 223.92258778],
       [159.45596423, 215.35110349, 160.129774  , ..., 141.24777578,
        223.92258778,   0.        ]])

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
mimport scipy.cluster.hierarchy as sch
        model = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
        model.fit(X)
        labels = model.labels_

In [39]:
import numpy as np
from scipy.spatial import distance_matrix
from matplotlib import pyplot as plt
#Agglomerative hierarchical clustering 
def k_means(new_array,K):
        nrow = new_array.shape[0]
        ncol = new_array.shape[1]
        
        # pick K random data points as intital centroids
        initial_centriods = np.random.choice(nrow, K, replace=False)
        centeroids = new_array[initial_centroids]
        centroids_old = np.zeros[K, ncol]
        cluster_assignments = np.zeros(nrows)
        while (centroids_old != centeroids).any():
            centeroids_old.append(centeroids)
            #compute the distances between data points and the centeriods 
            dist_matrix = distance_matrix(new_array, centeroids, p=2)
            #find closest centeroid
            for i in np.arrange(nrow): 
                d=dist_matrix[i]
                closest_centeroid = (np.where(d == np.min(d)))[0][0]
            #associate data points with closest centeriod
            cluster_assignments[i] = closest_centeroid
            #recompute centeriods
            for k in np.arange[K]:
                new_array_k = new_array[cluster_assignments==k]
                centeroids[k] = np.apply_along_axis(np.mean, axis=0, arr=new_array_k)
        return(centeroids, cluster_assignments)
            


In [42]:
# K Means Clustering 
#Randomly intitalize two data points called the center centeroids
#Use Euclidean distacne to find what data point is closest to the centeroids
#Based on the distance from c1 and c2 centeroids, the data point will be grouped into clusters
#Compute the datapoints of the centeroid inside cluster 1 
#Repostion the centeroid of the cluster 1 to the new centeroid
#Compute the centeroid of datapoints inside cluster 2
#Reposition the centeroid of cluster 2 to the new centeroid 
#Repeat the calculation of centeroids and repostioning until none of the cluster assignments change

In [ ]:
#The similiarity metric I used is called Euclidean distance: I used it because to assign data points to a centeroid I needed a proximity measurement. 
#Euclidean was the best option for me biologically because I can see if the backbone coordinates align, are the same close in distance. 
#In the end when thinking about using the backbone coordinates as a way to measure similarity in active site. 
#This was a decision biologically. biologically it would be hard to overlapped coordinates because all the active sites are not the same length in atoms. 
#For future directions it would be better if I chose either amino acid count or a count postive, negative , hydrophobic phenotype. These measurements would be more informative biologically. 
#I chose K-means clustering
#I chose agglomerative hierarchical clustering 